In [11]:
def generate_tables():
    polynomial = 0x43 # 2**8 + 2**4 + 2**3 + 2**2 + 1 =  1 0001 1101 binary
    s = 6
    gf_table_size = 1 << s # 1<<8 = 0x100 = 256 = 2**8

    # Initialize 
    gflog = gf_table_size * [0]
    gfilog = gf_table_size * [0]

    b = 1
    for i in range(0, gf_table_size):
        gflog[b] = i    # & 255
        gfilog[i] = b   # & 255 
        b <<= 1
        # Following line is could also be expressed as "if b >= gf_table_size"
        if b & gf_table_size:
            b ^= polynomial

    gflog[1] = 0; # Q for students: Why do you think is this needed? Try printing it out and see.
    return (gflog, gfilog)

def dump_table(caption, tab):
    item = 0
    print("--- {} ---".format(caption))
    for i in tab:
        print("0x{:02x}, ".format(i), end="")
        item += 1
        if item % 16 == 0:
            item = 0
            print()
    print("")

def qdump(tab):
    item = 0
    for i in tab:
        print("0x{:02x}, ".format(i), end="")
        item += 1
        if item % 16 == 0:
            item = 0
            print()
    print("")

(gflog, gfilog) = generate_tables()


In [2]:
dump_table("gflog",gflog)
dump_table("gfilog",gfilog)

--- gflog ---
0x00, 0x00, 0x01, 0x19, 0x02, 0x32, 0x1a, 0x1e, 0x03, 0x37, 0x33, 0x13, 0x1b, 0x09, 0x1f, 0x0c, 
0x04, 0x25, 0x38, 0x2e, 0x34, 0x3c, 0x14, 0x22, 0x1c, 0x17, 0x0a, 0x11, 0x20, 0x2c, 0x0d, 0x0f, 
0x05, 0x28, 0x26, 0x27, 0x39, 0x29, 0x2f, 0x06, 0x35, 0x07, 0x3d, 0x30, 0x15, 0x2a, 0x23, 0x3a, 
0x1d, 0x31, 0x18, 0x3e, 0x0b, 0x08, 0x12, 0x36, 0x21, 0x3b, 0x2d, 0x24, 0x0e, 0x2b, 0x10, 0x16, 

--- gfilog ---
0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x27, 0x29, 0x35, 0x0d, 0x1a, 0x34, 0x0f, 0x1e, 0x3c, 0x1f, 
0x3e, 0x1b, 0x36, 0x0b, 0x16, 0x2c, 0x3f, 0x19, 0x32, 0x03, 0x06, 0x0c, 0x18, 0x30, 0x07, 0x0e, 
0x1c, 0x38, 0x17, 0x2e, 0x3b, 0x11, 0x22, 0x23, 0x21, 0x25, 0x2d, 0x3d, 0x1d, 0x3a, 0x13, 0x26, 
0x2b, 0x31, 0x05, 0x0a, 0x14, 0x28, 0x37, 0x09, 0x12, 0x24, 0x2f, 0x39, 0x15, 0x2a, 0x33, 0x01, 



In [12]:
# A check to prove the tables really are inverses of each other (almost)
gf_num_elements = 0x40
for i in range (0, gf_num_elements):
    a = gflog[i]
    b = gfilog[i]
    if gfilog[a] != i:
        print ("gflog[0x{:02x}]  is 0x{:02x}, but gfilog[0x{:02x}] is 0x{:02x}".format(i,a,a,gfilog[a]))
    if gflog[b] != i:
        print ("gfilog[0x{:02x}] is 0x{:02x}, but gflog[0x{:02x}]  is 0x{:02x}".format(i,b,b,gflog[b]))

gflog[0x00]  is 0x00, but gfilog[0x00] is 0x01
gfilog[0x3f] is 0x01, but gflog[0x01]  is 0x00


In [14]:
# rs_functions.py

#from gflog_tables import *

# Addition
# Under a Galois Field 2^n, addition is always a simple XOR.
def gf_add(*args):
    result = 0
    for arg in args:
        result ^= arg

    return result

# Multiplication
def gf_mul(a, b):
    global gflog
    global gfilog

    if a == 0 or b == 0:
        return 0
    else:
        return gfilog[(gflog[a] + gflog[b]) % 63]

# Subtraction (Division helper)
def sub_gf8(a, b):
    if a > b:
        return a - b
    else:
        return (63 - (0 - (a - b)))  # Weird way to do modulus. 

# Division
def gf_div(a, b):
    global gfilog
    global gflog

    return gfilog[sub_gf8(gflog[a], gflog[b])]


In [13]:
#Print a portion of the gf(256) Multiplication Table
startx=0x0
starty=0x0

numrows=32
numcols=32

endx=startx+numcols
endy=starty+numrows

print ("   |", end="")
for x in range(startx,endx):
    print(" {:02x}".format(x), end="")
print("")
print ("---+", end="")
for x in range(startx,endx):
    print("---",end="")
print("")

for y in range(starty,endy):
    print("{:02x} |".format(y), end="")
    for x in range(startx,endx):
        print(" {:02x}".format(gf_mul(x,y)), end="")
    print("")
        


   | 00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f 10 11 12 13 14 15 16 17 18 19 1a 1b 1c 1d 1e 1f
---+------------------------------------------------------------------------------------------------
00 | 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
01 | 00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f 10 11 12 13 14 15 16 17 18 19 1a 1b 1c 1d 1e 1f
02 | 00 02 04 06 08 0a 0c 0e 10 12 14 16 18 1a 1c 1e 20 22 24 26 28 2a 2c 2e 30 32 34 36 38 3a 3c 3e
03 | 00 03 06 05 0c 0f 0a 09 18 1b 1e 1d 14 17 12 11 30 33 36 35 3c 3f 3a 39 28 2b 2e 2d 24 27 22 21
04 | 00 04 08 0c 10 14 18 1c 20 24 28 2c 30 34 38 3c 03 07 0b 0f 13 17 1b 1f 23 27 2b 2f 33 37 3b 3f
05 | 00 05 0a 0f 14 11 1e 1b 28 2d 22 27 3c 39 36 33 13 16 19 1c 07 02 0d 08 3b 3e 31 34 2f 2a 25 20
06 | 00 06 0c 0a 18 1e 14 12 30 36 3c 3a 28 2e 24 22 23 25 2f 29 3b 3d 37 31 13 15 1f 19 0b 0d 07 01
07 | 00 07 0e 09 1c 1b 12 15 38 3f 36 31 24 23 2a 2d 33 34 3d 3a 2f 28 21 26 0b 0c 05 02 17

In [15]:

print ("gflog(n) n   gfilog(n)  1/gfilog(n)")
for index in range(64):
    print ("{:02x}       {:02x}  {:02x}         {:02x}".format(gflog[index],index, gfilog[index],gf_div(1,gfilog[index])))
    

gflog(n) n   gfilog(n)  1/gfilog(n)
00       00  01         01
00       01  02         21
01       02  04         31
06       03  08         39
02       04  10         3d
0c       05  20         3f
07       06  03         3e
1a       07  06         1f
03       08  0c         2e
20       09  18         17
0d       0a  30         2a
23       0b  23         15
08       0c  05         2b
30       0d  0a         34
1b       0e  14         1a
12       0f  28         0d
04       10  13         27
18       11  26         32
21       12  0f         19
10       13  1e         2d
0e       14  3c         37
34       15  3b         3a
24       16  35         1d
36       17  29         2f
09       18  11         36
2d       19  22         1b
31       1a  07         2c
26       1b  0e         16
1c       1c  1c         0b
29       1d  38         24
13       1e  33         12
38       1f  25         09
05       20  09         25
3e       21  12         33
19       22  24         38
0b       23  0b    